In [1]:
import pyodbc
import pandas as pd
import sqlalchemy
import urllib
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
from save import save
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
typecurve_df = pd.read_excel(r'C:\Users\bbell\OneDrive - CrownQuest Operating\Desktop\mcdowell curves.xlsx', sheet_name='8 well spacing')

In [3]:
oil_price = 50
gas_price = 3
opex = 7

oil_tax = 0.046
gas_tax = 0.075

wi = 1
nri = 0.75

lease_bonus = 16800000

sections = 26
spacing = 8

rig_pace = 20
well_cost = 6300000

initial_cash = 233200000
initial_debt = 0
initial_investment = 250000000

In [4]:
rig_schedule_df = pd.read_excel(r'C:\Users\bbell\OneDrive - CrownQuest Operating\Desktop\rig schedule.xlsx', sheet_name='scenario 1')


In [5]:
rig_df = pd.DataFrame()
for i in rig_schedule_df.index:
    rig_start = rig_schedule_df.iloc[i,1]
    rig_end = rig_schedule_df.iloc[i,2]

    #create list of all dates between start and end
    #need to do a string function generator for frequency
    rig_dates = pd.date_range(rig_start, rig_end, freq='10.14D')
    rig_eoms = rig_dates + pd.offsets.Day() - pd.offsets.MonthEnd(-1)

    df = pd.DataFrame({
        'Well Count': 1,
        'Date': rig_eoms
        })

    rig_df = rig_df.append(df, ignore_index=True)

rig_df['Date'] = rig_df['Date'].dt.date
rig_df

,Well Count,Date
0,1,2023-01-31
1,1,2023-01-31
2,1,2023-02-28
3,1,2023-02-28
4,1,2023-02-28
...,...,...
205,1,2028-09-30
206,1,2028-09-30
207,1,2024-01-31
208,1,2025-01-31


In [6]:
wellcount_df = rig_df.groupby(['Date']).sum()
wellcount_df = wellcount_df.reset_index()
wellcount_df['Date'] = pd.to_datetime(wellcount_df['Date'])

print(wellcount_df['Well Count'].sum())

210


In [7]:
prod_df = pd.DataFrame()
final_df = pd.DataFrame()
for i in rig_df.index:
    date = rig_df.iloc[i, 1]

    df = pd.DataFrame({
        'Date': pd.date_range(date, periods=len(typecurve_df.index), freq='M'),
        'Oil': typecurve_df['oil'],
        'Gas': typecurve_df['gas']
        })
    prod_df = prod_df.append(df, ignore_index=True)

prod_df

,Date,Oil,Gas
0,2023-01-31,12961.126826,15801.505242
1,2023-02-28,12528.182037,17334.922460
2,2023-03-31,9552.583804,15507.716075
3,2023-04-30,8271.578649,14912.853558
4,2023-05-31,6950.315440,13505.067829
...,...,...,...
91135,2058-10-31,91.082675,227.233186
91136,2058-11-30,93.465729,233.178432
91137,2058-12-31,89.823097,224.090788
91138,2059-01-31,92.173195,229.953817


In [8]:
final_df = prod_df.groupby(['Date']).sum()
final_df = final_df.reset_index()
final_df['Date'] = pd.to_datetime(final_df['Date'])
final_df = final_df.merge(wellcount_df, on='Date', how='left')
final_df['Month'] = final_df.index + 1
final_df['Net Oil'] = final_df['Oil'] * nri
final_df['Net Gas'] = final_df['Gas'] * nri
final_df['Net Oil Revenue'] = final_df['Net Oil'] * oil_price
final_df['Net Gas Revenue'] = final_df['Net Gas'] * gas_price
final_df['Total Revenue'] = final_df['Net Oil Revenue'] + final_df['Net Gas Revenue']
final_df['Total Gross Expense'] = final_df['Oil'] * opex
final_df['Total Net Expense'] = final_df['Total Gross Expense'] * wi
final_df['Production Taxes'] = (final_df['Net Oil Revenue'] * oil_tax) + (final_df['Net Gas Revenue'] * gas_tax)
final_df['Operating Income'] = (final_df['Net Oil Revenue'] + final_df['Net Gas Revenue']) - final_df['Total Net Expense'] - final_df['Production Taxes']
final_df['Gross CAPEX'] = final_df['Well Count'] * well_cost
final_df['Net CAPEX'] = final_df['Gross CAPEX'] * wi
final_df.fillna(0, inplace=True)
final_df['Disc NET CAPEX'] = final_df['Net CAPEX']/(1.1 ** (final_df['Month']/12))

final_df['Cash on Hand'] = np.nan
for i in final_df.index:
    if i == 0:
        final_df.at[i, 'Cash on Hand'] = initial_cash
    else:
        final_df.at[i, 'Cash on Hand'] = final_df.at[(i-1), 'Cash on Hand'] - final_df.at[i, 'Gross CAPEX'] + final_df.at[i, 'Operating Income']
final_df['Cash on Hand'] = final_df['Cash on Hand'].clip(lower=0)

final_df['Debt'] = np.nan
for i in final_df.index:
    if i == 0:
        final_df.at[i, 'Debt'] = initial_debt
    else:
        if final_df.at[i, 'Cash on Hand'] > 0:
            final_df.at[i, 'Debt'] = 0
        else:
            final_df.at[i, 'Debt'] = final_df.at[(i-1), 'Debt'] + (final_df.at[i, 'Operating Income'] - final_df.at[i, 'Net CAPEX'])

final_df['Interest Expense'] = 0
for i in final_df.index:
    if final_df.at[i, 'Debt'] == 0:
        final_df.at[i, 'Interest Expense'] = 0
    else:
        final_df.at[i, 'Interest Expense'] = (0.1 / 12) * final_df.at[i, 'Debt']

final_df['Cash Flow'] = final_df['Operating Income'] - final_df['Net CAPEX'] + final_df['Interest Expense']
final_df['Discounted Cash Flow'] = final_df['Cash Flow'] / (1.1 ** (final_df['Month']/12))
final_df['Net Debt/EBITAX'] = abs(final_df['Debt'] / (final_df['Total Revenue'] - final_df['Total Net Expense']))

final_df

,Date,Oil,Gas,Well Count,Month,Net Oil,Net Gas,Net Oil Revenue,Net Gas Revenue,Total Revenue,...,Operating Income,Gross CAPEX,Net CAPEX,Disc NET CAPEX,Cash on Hand,Debt,Interest Expense,Cash Flow,Discounted Cash Flow,Net Debt/EBITAX
0,2023-01-31,38883.380477,47404.515726,3.0,1,29162.535358,35553.386794,1.458127e+06,106660.160383,1.564787e+06,...,1.217530e+06,18900000.0,18900000.0,1.875048e+07,2.332000e+08,0.0,0,-1.768247e+07,-1.754258e+07,0.0
1,2023-02-28,76467.926587,99409.283105,3.0,2,57350.944940,74556.962329,2.867547e+06,223670.886986,3.091218e+06,...,2.407260e+06,18900000.0,18900000.0,1.860214e+07,2.167073e+08,0.0,0,-1.649274e+07,-1.623282e+07,0.0
2,2023-03-31,105125.677999,145932.431329,3.0,3,78844.258499,109449.323497,3.942213e+06,328347.970491,4.270561e+06,...,3.328713e+06,18900000.0,18900000.0,1.845498e+07,2.011360e+08,0.0,0,-1.557129e+07,-1.520465e+07,0.0
3,2023-04-30,129940.413947,190670.992003,3.0,4,97455.310460,143003.244002,4.872766e+06,429009.732007,5.301775e+06,...,4.135869e+06,18900000.0,18900000.0,1.830898e+07,1.863718e+08,0.0,0,-1.476413e+07,-1.430245e+07,0.0
4,2023-05-31,150791.360268,231186.195491,3.0,5,113093.520201,173389.646619,5.654676e+06,520168.939856,6.174845e+06,...,4.820178e+06,18900000.0,18900000.0,1.816414e+07,1.722920e+08,0.0,0,-1.407982e+07,-1.353163e+07,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,2064-06-30,1467.669415,3661.543722,0.0,498,1100.752061,2746.157791,5.503760e+04,8238.473374,6.327608e+04,...,4.985278e+04,0.0,0.0,0.000000e+00,1.367579e+09,0.0,0,4.985278e+04,9.547571e+02,0.0
498,2064-07-31,1190.778757,2970.756518,0.0,499,893.084067,2228.067388,4.465420e+04,6684.202165,5.133841e+04,...,4.044755e+04,0.0,0.0,0.000000e+00,1.367619e+09,0.0,0,4.044755e+04,7.685044e+02,0.0
499,2064-08-31,912.731668,2277.084251,0.0,500,684.548751,1707.813188,3.422744e+04,5123.439564,3.935088e+04,...,3.100304e+04,0.0,0.0,0.000000e+00,1.367650e+09,0.0,0,3.100304e+04,5.843984e+02,0.0
500,2064-09-30,642.612383,1603.190277,0.0,501,481.959287,1202.392707,2.409796e+04,3607.178122,2.770514e+04,...,2.182781e+04,0.0,0.0,0.000000e+00,1.367672e+09,0.0,0,2.182781e+04,4.081930e+02,0.0


In [9]:
final_df.to_excel(r'C:\Users\bbell\OneDrive - CrownQuest Operating\Desktop\mcdowell cash statement.xlsx')

In [10]:
summary_df = pd.DataFrame({
    'Total Cash Flow': final_df['Cash Flow'].sum(),
    'Total Discounted Cash Flow': final_df['Discounted Cash Flow'].sum(),
    'Max Net Debt/EBITAX': final_df['Net Debt/EBITAX'].max(),
    'ROR': (final_df.at[final_df.index.max(), 'Cash on Hand'] - initial_investment) / initial_investment,
    'NPV': final_df['Discounted Cash Flow'].sum(),
    'DROI': (final_df['Discounted Cash Flow'].sum() + final_df['Disc NET CAPEX'].sum()) / final_df['Disc NET CAPEX'].sum(),
}, index=[0])


In [11]:
ddf = final_df.copy()

def irr(y):
    return sum(ddf['Cash Flow'] / ((1 + y) ** (ddf['Month']/12)))

def goal_seek(_threshold):
    lower = 0.1
    upper = 0.5
    solve = (lower + upper)/2
    threshold = irr(solve)
    while abs(threshold) >= _threshold:
        print(f'Threshold is: {threshold}')
        if threshold > 0:
            lower = solve
            solve = (lower + upper)/2
        elif threshold < 0:
            upper = solve
            solve = (lower + upper)/2
        print(f'range is: {lower}  ----  {solve}   ----   {upper}')
        threshold = irr(solve)

    return solve

In [12]:
summary_df['IRR'] = goal_seek(1000)
summary_df


Threshold is: -187772262.33105767
range is: 0.1  ----  0.2   ----   0.3
Threshold is: -125620810.96730514
range is: 0.1  ----  0.15000000000000002   ----   0.2
Threshold is: -51881869.95354474
range is: 0.1  ----  0.125   ----   0.15000000000000002
Threshold is: 6725787.054442684
range is: 0.125  ----  0.1375   ----   0.15000000000000002
Threshold is: -24973120.344550423
range is: 0.125  ----  0.13125   ----   0.1375
Threshold is: -9780464.498302205
range is: 0.125  ----  0.128125   ----   0.13125
Threshold is: -1699543.3909105845
range is: 0.125  ----  0.1265625   ----   0.128125
Threshold is: 2469018.0739873475
range is: 0.1265625  ----  0.12734374999999998   ----   0.128125
Threshold is: 373845.39989304973
range is: 0.12734374999999998  ----  0.12773437499999998   ----   0.128125
Threshold is: -665555.4466132263
range is: 0.12734374999999998  ----  0.12753906249999997   ----   0.12773437499999998
Threshold is: -146533.69341634403
range is: 0.12734374999999998  ----  0.12744140624999

,Total Cash Flow,Total Discounted Cash Flow,Max Net Debt/EBITAX,ROR,NPV,DROI,IRR
0,1.070860e+09,8.883623e+07,7.156178,4.470726,8.883623e+07,1.087545,0.127484


In [13]:
#summary_df.to_excel(r'C:\Users\bbell\OneDrive - CrownQuest Operating\Desktop\mcdowell summary.xlsx')



